In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import numpy as np
import pydrake.solvers.mathematicalprogram as mp
import mpc.mpc_tools as mpc
import mpc.symbolic as sym

In [11]:
m = 1.
l = 1.
g = 10.
A = np.array([
    [0., 1.],
    [g/l, 0.]
])
B = np.array([
    [0.],
    [1/(m*l**2.)]
])
N = 5
t_s = .1
sys = mpc.DTLinearSystem.from_continuous(t_s, A, B)

x_max = np.array([[np.pi/6.],[np.pi/22./(N*t_s)]])
x_min = -x_max
u_max = np.array([[m*g*l*np.pi/8.]])
u_min = -u_max

Q = np.eye(A.shape[0])/100.
R = np.eye(B.shape[1])

In [12]:
dim = 2

prog = mp.MathematicalProgram()

x = prog.NewContinuousVariables(2, N, "x")
u = prog.NewContinuousVariables(1, N, "u")

for j in range(N - 1):
    x_next = sys.A.dot(x[:, j]) + sys.B.dot(u[:, j])
    for i in range(dim):
        prog.AddLinearConstraint(x[i, j + 1] == x_next[i])
        
for j in range(N):
    for i in range(x.shape[0]):
        prog.AddLinearConstraint(x[i, j] <= x_max[i])
        prog.AddLinearConstraint(x[i, j] >= x_min[i])
    for i in range(u.shape[0]):
        prog.AddLinearConstraint(u[i, j] <= u_max[i])
        prog.AddLinearConstraint(u[i, j] >= u_min[i])
        
for j in range(N):
    prog.AddQuadraticCost(x[:, j].T.dot(Q).dot(x[:, j]))
    prog.AddQuadraticCost(u[:, j].T.dot(R).dot(u[:, j]))

In [13]:
qp = sym.CanonicalMPCQP.from_mathematicalprogram(prog, u, x)

In [14]:
qp.G

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.00504181,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.00504181,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.10167502,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.10167502,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.01563381,  0.00504181,  0.        ,  0.        ,  0.        ],
       [-0.01563381, -0.00504181,  0. 

In [16]:
qp.F

array([[ 0.03642823,  0.02752356,  0.01934582,  0.0106762 ,  0.        ],
       [ 0.01595563,  0.01129956,  0.00749677,  0.0039411 ,  0.        ]])